## Intructions

The main goal of the mini-project is to build the database of restaurants, bars and various points of interest (POIs) in the area of your choice and find out which API has better coverage in the selected area. The APIs have limited number of requests for free, so start with the smaller area.

APIS:
- Foursquare
- Yelp
- *Stretch*: Google

Steps:
- pull the data about various POIs in the area through API. If you run out of requests for any of the APIs, don't worry, it's ok to use only sample data you already have or the POIs from the Yelp API. It's approach and process that counts.
- create own SQLite database and store the data about the POIs. Think about what will be the best structure of the database
- compare the results using SQL or Pandas (it's up to you:)) and see which API has a better coverage of the area.
- choose the top 10 POIs based on the popularity
- *(Stretch)* By implementing [travelling salesman problem (TSP)](https://en.wikipedia.org/wiki/Travelling_salesman_problem), how much time would it take to visit all of these places?

### Step 1: Foursquare API

Make a function using foursquare API that takes a location and outputs a table of POI in the area. 

*Note*: While foursquare has unlimited queries, try and only run function once

In [1]:
#import packages
import pandas as pd
import os
import requests as re
from IPython.display import JSON

In [2]:
#define api url elements
root_endpoint_four = "https://api.foursquare.com/v3/places/search"

#set api key
foursquare_key = os.environ["FOURSQUARE_API_KEY"]

headers_four = {
    "Accept": "application/json",
    "Authorization": foursquare_key
}

In [3]:
#custom dict of category ids and labels
four_cat_ids = {'Arts and Entertainment': 10000,
           'Business and Professional Services':11000,
           'Community and Government':12000,
           'Dining and Drinking': 13000,
           'Event': 14000,
           'Health and Medicine': 15000,
           'Landmarks and Outdoors': 16000,
           'Retail': 17000,
           'Sports and Recreation': 18000,
           'Travel and Transportation': 19000
          }

In [4]:
#make a function that returns venue names for all venue types and total venue amount
def get_venues_four(location, venue_type):
    """ intakes: string of lat, long location 
        returns: int representing how many bars are within 5km of the crash
    ex: get_venues('45.6387,-122.6615') -> 8"""
    
    #get data relevent to input location
    response = re.get(root_endpoint_four, 
                      params={'ll': location, 'radius': 1000, 'categories': four_cat_ids[venue_type], 'limit': 50}, 
                      headers = headers_four)
    result = response.json()
    
    #define empty objects
    category_amount = 0
    venues = {'id': [],
              'title': []
    }
    
    #cycle into data to get place type
    for place in result['results']:
        
        #go into every type a place is
        for p_type in place['categories']:
    
            category_amount +=1 
            venues['title'].append(p_type['name'])
            venues['id'].append(p_type['id'])

    #save venues into data frame and count values
    venues_df = pd.DataFrame(venues).value_counts()
    #return a database of the venue names and an integer of total tagged with that id
    return pd.DataFrame(venues_df, columns=['total'])

test_df = get_venues_four('45.6387,-122.6615', 'Dining and Drinking')
test_df.head()

,,total
id,title,
13065,Restaurant,14
13068,American Restaurant,6
13031,Burger Joint,5
13002,Bakery,5
13035,Coffee Shop,4


### Yelp API

Since this the first time working with it let's start by setting a test result and looking at the return

In [5]:
#define api url elements
root_endpoint_yelp = "https://api.yelp.com/v3/businesses/search"

#set api key
yelp_key = os.environ["YELP_API"]

headers_yelp = {
    "Authorization": f"Bearer {yelp_key}"
}

In [6]:
#Test out retrieving API info
response = re.get(root_endpoint_yelp, 
                  params = {'latitude': 45.6387, 
                            'longitude':-122.6615,
                            'categories':'arts, All'},
                  headers = headers_yelp)

#make sure it works
print(response.status_code)

#convert into json (nested list of dictionaries)
result = response.json()

200


In [7]:
#let's explore this result
JSON(result['businesses'])

<IPython.core.display.JSON object>

In [8]:
#larger categories given by yelp
#yelp_cat_ids = {'Active Life': 'active, All',

In [9]:
def get_venues_yelp(location, category):
    
    #Test out retrieving API info
    response = re.get(root_endpoint_yelp, 
                  params = {'latitude': location[0], 
                            'longitude':location[1],
                            'categories':category,
                            'radius':1000},
                  headers = headers_yelp)
    
    result = response.json()
    
    #define empty objects
    venue_amount = 0
    venues = []
    
    for place in result['businesses']:
      
        #print(i['categories'])
        for venue_type in place['categories']:
        
            #print(venue_type['alias'])
            venues.append(venue_type['title'])
            venue_amount += 1
        

    #save venues into data frame and count values
    venues_df = pd.DataFrame(venues, columns=['title']).value_counts()
    #make dataframe look nice with proper index
    return  pd.DataFrame(venues_df, columns=['total'])
    
    
test_yelp_df = get_venues_yelp([45.6387,-122.6615], ['restaurants, All'])

In [10]:
print(test_yelp_df)
test_yelp_df.head(20)

                        total
title                        
American (Traditional)      5
Breakfast & Brunch          5
Food Trucks                 4
Coffee & Tea                3
Sandwiches                  3
American (New)              2
Beer, Wine & Spirits        2
Pubs                        2
Pizza                       1
Sushi Bars                  1
Speakeasies                 1
Mexican                     1
Tapas/Small Plates          1
Thai                        1
Vietnamese                  1
Waffles                     1
Noodles                     1
Italian                     1
Mediterranean               1
Japanese                    1
Hot Dogs                    1
French                      1
Delis                       1
Cocktail Bars               1
Chinese                     1
Burgers                     1
Brewpubs                    1
Bars                        1
Bakeries                    1
Wine Bars                   1


,total
title,
American (Traditional),5
Breakfast & Brunch,5
Food Trucks,4
Coffee & Tea,3
Sandwiches,3
American (New),2
"Beer, Wine & Spirits",2
Pubs,2
Pizza,1


### Larger data sets

I have the basics for generating API info. But I want to get venue amounts for a bunch of different categories and create one big table, one for foursquare, one for yelp.

Then table that will relate yelp venue titles and foursquare venue titles

In [11]:
#Let's define the foursquare venue list we'll be using
foursquare_cat = {'Arts and Entertainment': 10000,
                  'Dining and Drinking': 13000,
                  'Health and Medicine': 15000,
                  'Landmarks and Outdoors': 16000
                 }

yelp_cat = {'Arts and Entertainment': 'arts, All',
            'Dining and Drinking': 'restaurants, All',
            'Health and Medicine': 'health, All',
            'Landmarks and Outdoors': 'entertainment, All'
                 }

In [12]:
frames_four = []
#now i want a loop that will go in and generate a table for each of the dict elements
for category in foursquare_cat:
    
    frames_four.append(get_venues_four('45.6387,-122.6615', category))

In [13]:
#now let's put them all together
foursquare_raw = pd.concat([frames_four[0], frames_four[1], frames_four[2], frames_four[3]],
                            names=['Category','id','title'],
                            keys=['Entertainment','Dining','Health','Landmarks']).reset_index()
foursquare_raw.head()

,Category,id,title,total
0,Entertainment,10000,Arts and Entertainment,4
1,Entertainment,10039,Music Venue,2
2,Entertainment,10003,Arcade,1
3,Entertainment,10004,Art Gallery,1
4,Entertainment,10028,Art Museum,1


In [14]:
foursquare_data = foursquare_raw.sort_values('total', ascending = False)

In [15]:
#now it's yelps turn
frames_yelp = []
#now i want a loop that will go in and generate a table for each of the dict elements
for category in yelp_cat:
    frames_yelp.append(get_venues_yelp([45.6387, -122.6615], yelp_cat[category]))

In [16]:
yelp_raw = pd.concat([frames_yelp[0],frames_yelp[1],frames_yelp[2],frames_yelp[3]],
                      names=['Category','title'],
                      keys=['Entertainment','Dining','Health','Landmarks']).reset_index()


In [17]:
yelp_data = yelp_raw.sort_values('total', ascending = False)

In [18]:
yelp_data.to_csv('yelp_venues')

In [19]:
foursquare_data.to_csv('foursquare_venues')